In [2]:
from anndata import AnnData
import numpy as np
import pandas as pd
import h5py
import pyranges as pr
import scanpy as sc

In [4]:
data_read = "../data/for_python/"
analysis_save = "../analysis/"
data_save = "../data/for_python/"

# Read the H5 file
h5_file = 'multiome_normalized.h5'

with h5py.File(data_read+h5_file, 'r') as f:
    # Read the matrices
    rna_counts = f['rna_counts'][:]
    rna_normalized = f['rna_normalized'][:]
    atac_counts = f['atac_counts'][:]
    atac_normalized = f['atac_normalized'][:]
    
    # Read the names/identifiers
    rna_gene_names = f['rna_gene_names'][:]
    atac_feature_names = f['atac_feature_names'][:]
    cell_names = f['cell_names'][:]
    
    # Convert bytes to strings (HDF5 stores strings as bytes)
    if rna_gene_names.dtype.char == 'S':  # If stored as bytes
        rna_gene_names = [name.decode('utf-8') for name in rna_gene_names]
    
    if atac_feature_names.dtype.char == 'S':
        atac_feature_names = [name.decode('utf-8') for name in atac_feature_names]
    
    if cell_names.dtype.char == 'S':
        cell_names = [name.decode('utf-8') for name in cell_names]
    
    # Read metadata
    metadata = {}
    for key in f['metadata'].keys():
        data = f['metadata'][key][:]
        # Convert bytes to strings if needed
        if data.dtype.char == 'S':
            data = [item.decode('utf-8') for item in data]
        metadata[key] = data

In [5]:
rna_gene_names = np.array([x.decode('utf-8') if isinstance(x, bytes) else str(x) for x in rna_gene_names])
atac_feature_names = np.array([x.decode('utf-8') if isinstance(x, bytes) else str(x) for x in atac_feature_names])
cell_names = np.array([x.decode('utf-8') if isinstance(x, bytes) else str(x) for x in cell_names])


In [32]:
rna_gene_names = rna_gene_names.tolist()
atac_feature_names = atac_feature_names.tolist()
cell_names = cell_names.tolist()

In [25]:
# read genomic regions data
ann = pr.read_gtf("../data/Homo_sapiens.GRCh38.102.gtf")

In [51]:
ann_select = ann[ann.Feature == 'gene']
ann_select = ann_select[ann_select.gene_name.isin(rna_gene_names)]
ann_select = ann_select[[ 'gene_name']]
ann_select = ann_select.extend({'5': 3000})
ann_select[ann_select.Start < 0]

In [35]:
filtered_genes = set(rna_gene_names).intersection(set(ann_select.gene_name.values))

In [42]:
# convert genomic regions
atac_feature_names_pr = [[name.split('-')[0], name.split('-')[1], name.split('-')[2]] for name in atac_feature_names]
atac_feature_names_pr = pd.DataFrame(atac_feature_names_pr, columns=['Chromosome', 'Start', 'End'])
atac_feature_names_pr = pr.PyRanges(atac_feature_names_pr)